In [1]:
# Import required libraries
import os
import sqlite3
import pandas as pd
from datetime import datetime
from typing import Dict, Any, Optional, List, TypedDict, Annotated
import google.generativeai as genai
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv

load_dotenv()

# Set your Gemini API key here
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

print("✅ All libraries imported successfully")
print(f"📍 Working directory: {os.getcwd()}")

c:\Users\bhavy\Desktop\Career\Internship\Clovertex\GenAI Internship Assignment\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All libraries imported successfully
📍 Working directory: c:\Users\bhavy\Desktop\Career\Internship\Clovertex\GenAI Internship Assignment\submissions


In [2]:
"""
LANGGRAPH STATE DEFINITION
Defines the state object passed between agents in the workflow
"""

class WorkflowState(TypedDict):
    """State object for LangGraph workflow"""
    # Input parameters
    work_type: str
    description: str
    priority: int
    request_date: str
    
    # Agent 1 output
    work_id: str
    
    # Agent 2 output
    work_request: Dict[str, Any]
    required_specialty: str
    alternate_specialty: Optional[str]
    
    # Agent 3 output
    primary_candidates: List[Dict[str, Any]]
    alternate_candidates: List[Dict[str, Any]]
    
    # Agent 4 output
    best_candidate: Dict[str, Any]
    all_candidates: List[Dict[str, Any]]
    
    # Agent 5 output
    final_result: Dict[str, Any]

print("✅ WorkflowState defined")

✅ WorkflowState defined


In [3]:
"""
DATABASE MANAGER CLASS
Handles all SQLite database operations with enhanced error handling
"""

class DatabaseManager:
    """Handles all database operations for the work allocation system"""
    
    def __init__(self, db_path: str = "radiology.db"):
        self.db_path = db_path
        self.conn = None
    
    def connect(self):
        """Establish database connection with foreign key enforcement"""
        self.conn = sqlite3.connect(self.db_path)
        self.conn.row_factory = sqlite3.Row
        # Enable foreign key constraints
        self.conn.execute("PRAGMA foreign_keys = ON")
        return self.conn
    
    def close(self):
        """Close database connection"""
        if self.conn:
            self.conn.close()
    
    def setup_database(self, data_folder: str = "data/"):
        """Create tables with foreign keys and import CSV data"""
        self.connect()
        cursor = self.conn.cursor()
        
        print("🔧 Setting up database with enhanced schema...")
        
        # Create work_requests table with foreign keys
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS work_requests (
                work_id TEXT PRIMARY KEY,
                work_type TEXT NOT NULL,
                description TEXT,
                priority INTEGER CHECK(priority >= 1 AND priority <= 5),
                timestamp DATETIME,
                status TEXT DEFAULT 'pending',
                assigned_to TEXT,
                FOREIGN KEY (assigned_to) REFERENCES resources(resource_id)
                    ON DELETE SET NULL
            )
        """)
        
        # Create resources table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS resources (
                resource_id TEXT PRIMARY KEY,
                name TEXT NOT NULL,
                specialty TEXT NOT NULL,
                skill_level INTEGER CHECK(skill_level >= 1 AND skill_level <= 5),
                total_cases_handled INTEGER DEFAULT 0
            )
        """)
        
        # Create resource_calendar table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS resource_calendar (
                calendar_id TEXT PRIMARY KEY,
                resource_id TEXT NOT NULL,
                date DATE NOT NULL,
                available_from TIME NOT NULL,
                available_to TIME NOT NULL,
                current_workload INTEGER DEFAULT 0,
                FOREIGN KEY (resource_id) REFERENCES resources(resource_id)
                    ON DELETE CASCADE
            )
        """)
        
        # Create specialty_mapping table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS specialty_mapping (
                work_type TEXT PRIMARY KEY,
                required_specialty TEXT NOT NULL,
                alternate_specialty TEXT
            )
        """)
        
        # Create indexes
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_resources_specialty ON resources(specialty)")
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_calendar_resource ON resource_calendar(resource_id)")
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_work_status ON work_requests(status)")
        
        self.conn.commit()
        
        # Import CSV data
        resources_df = pd.read_csv(os.path.join(data_folder, "resources.csv"))
        resources_df.to_sql("resources", self.conn, if_exists="replace", index=False)
        print(f" ✓ Imported {len(resources_df)} resources")
        
        calendar_df = pd.read_csv(os.path.join(data_folder, "resource_calendar.csv"))
        calendar_df.to_sql("resource_calendar", self.conn, if_exists="replace", index=False)
        print(f" ✓ Imported {len(calendar_df)} calendar entries")
        
        specialty_df = pd.read_csv(os.path.join(data_folder, "specialty_mapping.csv"))
        specialty_df.to_sql("specialty_mapping", self.conn, if_exists="replace", index=False)
        print(f" ✓ Imported {len(specialty_df)} specialty mappings")
        
        work_df = pd.read_csv(os.path.join(data_folder, "work_requests.csv"))
        work_df.to_sql("work_requests", self.conn, if_exists="replace", index=False)
        print(f" ✓ Imported {len(work_df)} work requests")
        
        self.conn.commit()
        print("✅ Database setup complete with foreign key constraints!")
    
    def add_work_request(self, work_type: str, description: str, priority: int, custom_date: str = None) -> str:
        """Add a new work request with input validation"""
        # Validate priority
        if not isinstance(priority, int) or not 1 <= priority <= 5:
            raise ValueError("Priority must be integer between 1 and 5")
        
        cursor = self.conn.cursor()
        cursor.execute("SELECT COUNT(*) as count FROM work_requests")
        count = cursor.fetchone()[0]
        work_id = f"W{str(count + 1).zfill(3)}"
        
        if custom_date:
            timestamp = f"{custom_date} {datetime.now().strftime('%H:%M:%S')}"
        else:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        cursor.execute("""
            INSERT INTO work_requests (work_id, work_type, description, priority, timestamp, status)
            VALUES (?, ?, ?, ?, ?, 'pending')
        """, (work_id, work_type, description, priority, timestamp))
        self.conn.commit()
        return work_id
    
    def get_work_request(self, work_id: str) -> Optional[Dict[str, Any]]:
        """Get work request details by ID"""
        cursor = self.conn.cursor()
        cursor.execute("SELECT * FROM work_requests WHERE work_id = ?", (work_id,))
        row = cursor.fetchone()
        return dict(row) if row else None
    
    def get_specialty_mapping(self, work_type: str) -> Optional[Dict[str, str]]:
        """Get required and alternate specialties for a work type"""
        cursor = self.conn.cursor()
        cursor.execute("SELECT * FROM specialty_mapping WHERE work_type = ?", (work_type,))
        row = cursor.fetchone()
        return dict(row) if row else None
    
    def find_resources_by_specialty(self, specialty: str) -> List[Dict[str, Any]]:
        """Find all resources with given specialty"""
        cursor = self.conn.cursor()
        cursor.execute("SELECT * FROM resources WHERE specialty = ?", (specialty,))
        return [dict(row) for row in cursor.fetchall()]
    
    def get_resource_availability(self, resource_id: str, date: str) -> Optional[Dict[str, Any]]:
        """Get availability info for a resource on a specific date"""
        cursor = self.conn.cursor()
        cursor.execute("""
            SELECT * FROM resource_calendar 
            WHERE resource_id = ? AND date = ?
        """, (resource_id, date))
        row = cursor.fetchone()
        return dict(row) if row else None
    
    def assign_work(self, work_id: str, resource_id: str) -> bool:
        """Enhanced assignment with transaction management"""
        cursor = self.conn.cursor()
        
        try:
            cursor.execute("BEGIN TRANSACTION")
            
            # Validate work not already assigned
            work = self.get_work_request(work_id)
            if not work:
                raise ValueError(f"Work request {work_id} not found")
            if work['status'] == 'assigned':
                raise ValueError(f"Work {work_id} already assigned to {work['assigned_to']}")
            
            # Update work_requests
            cursor.execute('''
                UPDATE work_requests 
                SET assigned_to = ?, status = 'assigned'
                WHERE work_id = ?
            ''', (resource_id, work_id))
            
            # INCREMENT total_cases_handled (FIX #1)
            cursor.execute('''
                UPDATE resources 
                SET total_cases_handled = total_cases_handled + 1
                WHERE resource_id = ?
            ''', (resource_id,))
            
            self.conn.commit()
            return True
            
        except Exception as e:
            self.conn.rollback()
            print(f"❌ Assignment failed: {e}")
            raise
    
    def update_workload(self, resource_id: str, date: str, increment: int = 1):
        """Update resource workload for a specific date"""
        cursor = self.conn.cursor()
        cursor.execute("""
            UPDATE resource_calendar
            SET current_workload = current_workload + ?
            WHERE resource_id = ? AND date = ?
        """, (increment, resource_id, date))
        self.conn.commit()

print("✅ DatabaseManager class defined with enhanced features")

✅ DatabaseManager class defined with enhanced features


In [4]:
"""
RESOURCE SCORER
Transparent 100-point weighted scoring system
"""

class ResourceScorer:
    """Calculate scores for resource candidates based on multiple factors"""
    
    WEIGHTS = {
        'skill_level': 25,
        'experience': 20,
        'availability': 30,
        'workload': 15,
        'priority_bonus': 10
    }
    
    def calculate_score(self, resource: Dict[str, Any], availability: Dict[str, Any],
                       work_request: Dict[str, Any], role_match_type: str) -> Dict[str, Any]:
        """Calculate comprehensive score for a resource candidate"""
        
        if role_match_type == 'none':
            return {'total_score': 0, 'breakdown': {}, 'reason': 'No specialty match'}
        
        # Calculate individual scores
        skill_score = (resource['skill_level'] / 5.0) * self.WEIGHTS['skill_level']
        experience_score = min(resource['total_cases_handled'] / 20.0, self.WEIGHTS['experience'])
        
        # Availability score
        available_from = datetime.strptime(availability['available_from'], '%H:%M:%S').time()
        available_to = datetime.strptime(availability['available_to'], '%H:%M:%S').time()
        from_minutes = available_from.hour * 60 + available_from.minute
        to_minutes = available_to.hour * 60 + available_to.minute
        hours_available = (to_minutes - from_minutes) / 60.0
        availability_score = min(hours_available / 8.0, 1.0) * self.WEIGHTS['availability']
        
        # Workload score (lower is better)
        workload_score = max(0, self.WEIGHTS['workload'] - (availability['current_workload'] * 1.5))
        
        # Priority bonus
        if work_request['priority'] >= 4 and availability['current_workload'] <= 2:
            priority_bonus = self.WEIGHTS['priority_bonus']
        else:
            priority_bonus = (work_request['priority'] / 5.0) * self.WEIGHTS['priority_bonus']
        
        # Calculate total
        total_score = skill_score + experience_score + availability_score + workload_score + priority_bonus
        
        # Apply multiplier for alternate specialty
        multiplier = 0.8 if role_match_type == 'alternate' else 1.0
        total_score *= multiplier
        
        return {
            'total_score': round(total_score, 2),
            'breakdown': {
                'skill_score': round(skill_score, 2),
                'experience_score': round(experience_score, 2),
                'availability_score': round(availability_score, 2),
                'workload_score': round(workload_score, 2),
                'priority_bonus': round(priority_bonus, 2),
                'role_match_type': role_match_type,
                'multiplier': multiplier
            }
        }
    
    def rank_candidates(self, candidates: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Sort candidates by total score (descending)"""
        return sorted(candidates, key=lambda x: x['score']['total_score'], reverse=True)

print("✅ ResourceScorer class defined")

✅ ResourceScorer class defined


In [5]:
"""
LANGGRAPH AGENT NODE FUNCTIONS
Each function represents one agent in the workflow
"""

# Agent 1: Add Work Request
def add_work_node(state: WorkflowState, db: DatabaseManager) -> WorkflowState:
    """Agent 1: Add work request to database"""
    print(f"\n{'='*60}")
    print("[AGENT 1: AddWorkAgent] Processing new work request")
    print('='*60)
    print(f" Work Type: {state['work_type']}")
    print(f" Description: {state['description']}")
    print(f" Priority: {state['priority']}/5")
    print(f" Request Date: {state['request_date']}")
    
    work_id = db.add_work_request(
        state['work_type'], 
        state['description'], 
        state['priority'], 
        state['request_date']
    )
    print(f" ✅ Work request created: {work_id}")
    
    state['work_id'] = work_id
    return state

# Agent 2: Analyze Work
def analyze_work_node(state: WorkflowState, db: DatabaseManager) -> WorkflowState:
    """Agent 2: Analyze work request and determine required specialty"""
    print(f"\n{'='*60}")
    print(f"[AGENT 2: WorkAnalyzerAgent] Analyzing {state['work_id']}")
    print('='*60)
    
    work_request = db.get_work_request(state['work_id'])
    if not work_request:
        raise ValueError(f"Work request {state['work_id']} not found")
    
    print(f" Work Type: {work_request['work_type']}")
    print(f" Priority: {work_request['priority']}/5")
    
    specialty_map = db.get_specialty_mapping(work_request['work_type'])
    if not specialty_map:
        raise ValueError(f"No specialty mapping for {work_request['work_type']}")
    
    required_specialty = specialty_map['required_specialty']
    alternate_specialty = specialty_map.get('alternate_specialty')
    
    print(f" Required Specialty: {required_specialty}")
    if alternate_specialty:
        print(f" Alternate Specialty: {alternate_specialty}")
    print(f" ✅ Analysis complete")
    
    state['work_request'] = work_request
    state['required_specialty'] = required_specialty
    state['alternate_specialty'] = alternate_specialty
    return state

# Agent 3: Find Resources
def find_resources_node(state: WorkflowState, db: DatabaseManager) -> WorkflowState:
    """Agent 3: Find resources matching required specialty"""
    print(f"\n{'='*60}")
    print("[AGENT 3: ResourceFinderAgent] Finding matching resources")
    print('='*60)
    
    primary_candidates = db.find_resources_by_specialty(state['required_specialty'])
    print(f" Primary ({state['required_specialty']}): {len(primary_candidates)} found")
    for candidate in primary_candidates:
        print(f" • {candidate['name']} (Skill: {candidate['skill_level']}/5)")
    
    alternate_candidates = []
    if state.get('alternate_specialty'):
        alternate_candidates = db.find_resources_by_specialty(state['alternate_specialty'])
        print(f" Alternate ({state['alternate_specialty']}): {len(alternate_candidates)} found")
        for candidate in alternate_candidates:
            print(f" • {candidate['name']} (Skill: {candidate['skill_level']}/5)")
    
    print(f" ✅ Total: {len(primary_candidates) + len(alternate_candidates)} candidates")
    
    state['primary_candidates'] = primary_candidates
    state['alternate_candidates'] = alternate_candidates
    return state

# Agent 4: Score and Select
def score_candidates_node(state: WorkflowState, db: DatabaseManager) -> WorkflowState:
    """Agent 4: Score candidates and select best match"""
    print(f"\n{'='*60}")
    print("[AGENT 4: AvailabilityCheckerAgent] Scoring candidates")
    print('='*60)
    
    scorer = ResourceScorer()
    all_scored = []
    request_date = state['work_request']['timestamp'].split()[0]
    print(f" Request Date: {request_date}")
    
    # Score primary candidates
    print(f"\n Scoring primary candidates...")
    for resource in state['primary_candidates']:
        availability = db.get_resource_availability(resource['resource_id'], request_date)
        if availability:
            score = scorer.calculate_score(resource, availability, state['work_request'], 'exact')
            all_scored.append({
                'resource': resource,
                'availability': availability,
                'score': score
            })
            print(f" • {resource['name']}: {score['total_score']}/100")
    
    # Score alternate candidates
    if state.get('alternate_candidates'):
        print(f"\n Scoring alternate candidates...")
        for resource in state['alternate_candidates']:
            availability = db.get_resource_availability(resource['resource_id'], request_date)
            if availability:
                score = scorer.calculate_score(resource, availability, state['work_request'], 'alternate')
                all_scored.append({
                    'resource': resource,
                    'availability': availability,
                    'score': score
                })
                print(f" • {resource['name']}: {score['total_score']}/100 (alternate)")
    
    # Rank candidates
    ranked = scorer.rank_candidates(all_scored)
    if not ranked:
        raise ValueError("No suitable candidates found")
    
    best_candidate = ranked[0]
    print(f"\n 📊 Best match: {best_candidate['resource']['name']}")
    print(f" 📊 Score: {best_candidate['score']['total_score']}/100")
    print(f" ✅ Selection complete")
    
    state['best_candidate'] = best_candidate
    state['all_candidates'] = ranked
    return state

# Agent 5: Assign and Explain
def assign_work_node(state: WorkflowState, db: DatabaseManager, gemini_api_key: str) -> WorkflowState:
    """Agent 5: Finalize assignment and generate LLM explanation"""
    print(f"\n{'='*60}")
    print("[AGENT 5: AssignmentAgent] Finalizing assignment")
    print('='*60)
    
    resource = state['best_candidate']['resource']
    availability = state['best_candidate']['availability']
    score = state['best_candidate']['score']
    
    # Update database with enhanced logging (FIX #2)
    print(f" Updating database...")
    try:
        db.assign_work(state['work_id'], resource['resource_id'])
        db.update_workload(resource['resource_id'], availability['date'], 1)
        
        print(f" ✅ Updates completed:")
        print(f"    - Status: pending → assigned")
        print(f"    - Assigned to: {resource['resource_id']}")
        print(f"    - Total cases: +1")
        print(f"    - Workload: +1")
    except Exception as e:
        print(f" ❌ Database update failed: {e}")
        raise
    
    # Generate LLM explanation with enhanced prompt (FIX #3)
    print(f" Generating AI explanation...")
    genai.configure(api_key=gemini_api_key)
    model = genai.GenerativeModel('gemini-pro')
    
    prompt = f"""You are an expert medical staffing coordinator AI. Explain this assignment decision professionally.

ASSIGNMENT CONTEXT:
- Work: {state['work_request']['work_type']}
- Description: {state['work_request']['description']}
- Priority: {state['work_request']['priority']}/5 {"(URGENT)" if state['work_request']['priority'] >= 4 else "(Routine)"}

SELECTED RADIOLOGIST:
- Name: {resource['name']}
- Specialty: {resource['specialty']} (Match: {score['breakdown']['role_match_type']})
- Skill Level: {resource['skill_level']}/5
- Experience: {resource['total_cases_handled']} cases completed
- Availability: {availability['available_from']}-{availability['available_to']}
- Current Workload: {availability['current_workload']} active cases

DECISION SCORE: {score['total_score']}/100

Generate 2-3 sentences explaining why this radiologist is optimal for this case, highlighting key strengths.
Professional medical tone. No markdown formatting."""
    
    try:
        response = model.generate_content(prompt)
        explanation = response.text.strip()
    except Exception:
        # Fallback if API fails
        explanation = f"{resource['name']} was assigned based on {resource['specialty']} specialty, skill level {resource['skill_level']}/5, and {resource['total_cases_handled']} cases experience with current workload of {availability['current_workload']} cases."
    
    print(f" ✅ Explanation generated")
    
    print(f"\n {'='*60}")
    print(f" 🎯 ASSIGNMENT COMPLETE")
    print(f" {'='*60}")
    print(f" Work ID: {state['work_id']}")
    print(f" Assigned To: {resource['name']}")
    print(f" Score: {score['total_score']}/100")
    
    state['final_result'] = {
        'work_id': state['work_id'],
        'assigned_to': resource['resource_id'],
        'resource_name': resource['name'],
        'score': score,
        'explanation': explanation
    }
    
    return state

print("✅ All LangGraph agent nodes defined")

✅ All LangGraph agent nodes defined


In [6]:
"""
LANGGRAPH WORKFLOW CREATION
Creates the actual LangGraph state machine with proper orchestration
"""

def create_workflow(db: DatabaseManager, gemini_api_key: str) -> Any:
    """Create LangGraph workflow with 5 agents"""
    
    # Create StateGraph
    workflow = StateGraph(WorkflowState)
    
    # Add nodes (agents)
    workflow.add_node("add_work", lambda state: add_work_node(state, db))
    workflow.add_node("analyze", lambda state: analyze_work_node(state, db))
    workflow.add_node("find_resources", lambda state: find_resources_node(state, db))
    workflow.add_node("score", lambda state: score_candidates_node(state, db))
    workflow.add_node("assign", lambda state: assign_work_node(state, db, gemini_api_key))
    
    # Add edges (workflow order)
    workflow.add_edge("add_work", "analyze")
    workflow.add_edge("analyze", "find_resources")
    workflow.add_edge("find_resources", "score")
    workflow.add_edge("score", "assign")
    workflow.add_edge("assign", END)
    
    # Set entry point
    workflow.set_entry_point("add_work")
    
    return workflow.compile()

def run_workflow(db: DatabaseManager, gemini_api_key: str, 
                work_type: str, description: str, priority: int, 
                request_date: str = "2024-11-10") -> Dict[str, Any]:
    """Execute complete LangGraph workflow"""
    print("\n" + "="*80)
    print("🚀 STARTING LANGGRAPH WORKFLOW")
    print("="*80)
    
    try:
        # Create workflow
        app = create_workflow(db, gemini_api_key)
        
        # Initialize state
        initial_state: WorkflowState = {
            'work_type': work_type,
            'description': description,
            'priority': priority,
            'request_date': request_date,
            'work_id': '',
            'work_request': {},
            'required_specialty': '',
            'alternate_specialty': None,
            'primary_candidates': [],
            'alternate_candidates': [],
            'best_candidate': {},
            'all_candidates': [],
            'final_result': {}
        }
        
        # Run workflow
        final_state = app.invoke(initial_state)
        
        print("\n" + "="*80)
        print("✅ LANGGRAPH WORKFLOW COMPLETED SUCCESSFULLY")
        print("="*80)
        
        return {
            'success': True,
            'result': final_state['final_result'],
            'all_candidates': final_state['all_candidates']
        }
        
    except Exception as e:
        print(f"\n❌ Workflow failed: {str(e)}")
        import traceback
        traceback.print_exc()
        return {'success': False, 'error': str(e)}

print("✅ LangGraph workflow orchestrator defined")

✅ LangGraph workflow orchestrator defined


In [ ]:
# Initialize SYSTEM
db = DatabaseManager("radiology.db")
db.setup_database("data/")

print("\n🎯 System ready with true LangGraph orchestration!")

🔧 Setting up database with enhanced schema...
 ✓ Imported 15 resources
 ✓ Imported 45 calendar entries
 ✓ Imported 8 specialty mappings
 ✓ Imported 22 work requests
✅ Database setup complete with foreign key constraints!

🎯 System ready with true LangGraph orchestration!


### Test Scenarios (Using LangGraph)

In [8]:
"""
Scenario 1: Urgent Neurological Case (Priority 5)
Demonstrates: LangGraph orchestration, Priority handling, Exact specialty match
"""

result1 = run_workflow(
    db=db,
    gemini_api_key=GEMINI_API_KEY,
    work_type="MRI_Brain",
    description="Suspected acute stroke - immediate MRI evaluation required",
    priority=5
)

if result1['success']:
    print("\n" + "="*80)
    print("SCENARIO 1 RESULTS")
    print("="*80)
    
    res = result1['result']
    print(f"\n✅ Assignment: {res['resource_name']} ({res['assigned_to']})")
    print(f"\n📊 Score Breakdown:")
    for key, value in res['score']['breakdown'].items():
        print(f"   {key}: {value}")
    print(f"\n   TOTAL SCORE: {res['score']['total_score']}/100")
    
    print(f"\n💡 AI Explanation:")
    print(f"   {res['explanation']}")
    
    print(f"\n🏆 Top 3 Candidates:")
    for i, candidate in enumerate(result1['all_candidates'][:3], 1):
        print(f"   {i}. {candidate['resource']['name']}: {candidate['score']['total_score']}/100")


🚀 STARTING LANGGRAPH WORKFLOW

[AGENT 1: AddWorkAgent] Processing new work request
 Work Type: MRI_Brain
 Description: Suspected acute stroke - immediate MRI evaluation required
 Priority: 5/5
 Request Date: 2024-11-10
 ✅ Work request created: W023

[AGENT 2: WorkAnalyzerAgent] Analyzing W023
 Work Type: MRI_Brain
 Priority: 5/5
 Required Specialty: Neurologist
 Alternate Specialty: General_Radiologist
 ✅ Analysis complete

[AGENT 3: ResourceFinderAgent] Finding matching resources
 Primary (Neurologist): 4 found
 • Dr. Sarah Chen (Skill: 5/5)
 • Dr. James Wilson (Skill: 2/5)
 • Dr. Maria Garcia (Skill: 4/5)
 • Dr. Kevin Park (Skill: 4/5)
 Alternate (General_Radiologist): 4 found
 • Dr. John Smith (Skill: 4/5)
 • Dr. Emily Brown (Skill: 3/5)
 • Dr. Michael Davis (Skill: 3/5)
 • Dr. Alex Johnson (Skill: 4/5)
 ✅ Total: 8 candidates

[AGENT 4: AvailabilityCheckerAgent] Scoring candidates
 Request Date: 2024-11-10

 Scoring primary candidates...
 • Dr. Sarah Chen: 92.75/100
 • Dr. James Wi

In [9]:
"""
Scenario 2: Routine Chest X-Ray (Priority 2)
Demonstrates: Workload balancing, Lower priority handling
"""

result2 = run_workflow(
    db=db,
    gemini_api_key=GEMINI_API_KEY,
    work_type="X_Ray_Chest",
    description="Annual checkup - routine chest X-ray screening",
    priority=2,
    request_date="2024-11-11"
)

if result2['success']:
    print("\n" + "="*80)
    print("SCENARIO 2 RESULTS")
    print("="*80)
    
    res = result2['result']
    print(f"\n✅ Assignment: {res['resource_name']} ({res['assigned_to']})")
    print(f"\n📊 Score: {res['score']['total_score']}/100")
    print(f"\n💡 AI Explanation:")
    print(f"   {res['explanation']}")


🚀 STARTING LANGGRAPH WORKFLOW

[AGENT 1: AddWorkAgent] Processing new work request
 Work Type: X_Ray_Chest
 Description: Annual checkup - routine chest X-ray screening
 Priority: 2/5
 Request Date: 2024-11-11
 ✅ Work request created: W024

[AGENT 2: WorkAnalyzerAgent] Analyzing W024
 Work Type: X_Ray_Chest
 Priority: 2/5
 Required Specialty: General_Radiologist
 ✅ Analysis complete

[AGENT 3: ResourceFinderAgent] Finding matching resources
 Primary (General_Radiologist): 4 found
 • Dr. John Smith (Skill: 4/5)
 • Dr. Emily Brown (Skill: 3/5)
 • Dr. Michael Davis (Skill: 3/5)
 • Dr. Alex Johnson (Skill: 4/5)
 ✅ Total: 4 candidates

[AGENT 4: AvailabilityCheckerAgent] Scoring candidates
 Request Date: 2024-11-11

 Scoring primary candidates...
 • Dr. John Smith: 74.9/100
 • Dr. Emily Brown: 45.75/100
 • Dr. Michael Davis: 60.6/100
 • Dr. Alex Johnson: 53.6/100

 📊 Best match: Dr. John Smith
 📊 Score: 74.9/100
 ✅ Selection complete

[AGENT 5: AssignmentAgent] Finalizing assignment
 Updati

In [10]:
"""
Scenario 3: Specialized Mammography (Priority 3)
Demonstrates: Alternate specialty handling, 0.8x multiplier
"""

result3 = run_workflow(
    db=db,
    gemini_api_key=GEMINI_API_KEY,
    work_type="Mammography",
    description="Annual mammography screening for early detection",
    priority=3
)

if result3['success']:
    print("\n" + "="*80)
    print("SCENARIO 3 RESULTS")
    print("="*80)
    
    res = result3['result']
    print(f"\n✅ Assignment: {res['resource_name']} ({res['assigned_to']})")
    print(f"\n📊 Score: {res['score']['total_score']}/100")
    print(f"\n💡 AI Explanation:")
    print(f"   {res['explanation']}")
    
    print(f"\n🏆 All Candidates (Exact vs Alternate):")
    for i, candidate in enumerate(result3['all_candidates'], 1):
        match_type = candidate['score']['breakdown']['role_match_type']
        print(f"   {i}. {candidate['resource']['name']}: {candidate['score']['total_score']}/100 [{match_type}]")


🚀 STARTING LANGGRAPH WORKFLOW

[AGENT 1: AddWorkAgent] Processing new work request
 Work Type: Mammography
 Description: Annual mammography screening for early detection
 Priority: 3/5
 Request Date: 2024-11-10
 ✅ Work request created: W025

[AGENT 2: WorkAnalyzerAgent] Analyzing W025
 Work Type: Mammography
 Priority: 3/5
 Required Specialty: Breast_Imaging_Specialist
 Alternate Specialty: General_Radiologist
 ✅ Analysis complete

[AGENT 3: ResourceFinderAgent] Finding matching resources
 Primary (Breast_Imaging_Specialist): 2 found
 • Dr. Patricia Taylor (Skill: 3/5)
 • Dr. William Moore (Skill: 5/5)
 Alternate (General_Radiologist): 4 found
 • Dr. John Smith (Skill: 4/5)
 • Dr. Emily Brown (Skill: 3/5)
 • Dr. Michael Davis (Skill: 3/5)
 • Dr. Alex Johnson (Skill: 4/5)
 ✅ Total: 6 candidates

[AGENT 4: AvailabilityCheckerAgent] Scoring candidates
 Request Date: 2024-11-10

 Scoring primary candidates...
 • Dr. Patricia Taylor: 64.7/100
 • Dr. William Moore: 85.7/100

 Scoring altern

In [11]:
import sqlite3
import pandas as pd

def inspect_database_complete(db_path="radiology.db"):
    """Complete database inspection showing all tables and updates"""
    conn = sqlite3.connect(db_path)
    
    print("\n" + "="*80)
    print("🗄️  RADIOLOGY DATABASE INSPECTION")
    print("="*80)
    
    # Show work requests
    work_df = pd.read_sql_query("""
        SELECT work_id, work_type, priority, status, assigned_to, timestamp
        FROM work_requests
        ORDER BY work_id
    """, conn)
    print("\n📋 WORK REQUESTS:")
    print(work_df.to_string(index=False))
    
    # Show updated case counts
    resources_df = pd.read_sql_query("""
        SELECT resource_id, name, specialty, skill_level, total_cases_handled
        FROM resources
        ORDER BY total_cases_handled DESC
    """, conn)
    print("\n\n👨‍⚕️ RESOURCES (Updated Case Counts):")
    print(resources_df.to_string(index=False))
    
    # Show assignment details
    assignments_df = pd.read_sql_query("""
        SELECT w.work_id, w.work_type, r.name as assigned_to, r.total_cases_handled
        FROM work_requests w
        JOIN resources r ON w.assigned_to = r.resource_id
        WHERE w.status = 'assigned'
    """, conn)
    print("\n\n✅ ASSIGNMENTS:")
    print(assignments_df.to_string(index=False))
    
    conn.close()
    print("\n" + "="*80)

inspect_database_complete()



🗄️  RADIOLOGY DATABASE INSPECTION

📋 WORK REQUESTS:
work_id          work_type  priority    status assigned_to           timestamp
   W001      CT_Scan_Chest         4   pending        None 2024-11-12 07:17:00
   W002          MRI_Brain         4 completed        R011 2024-11-12 03:47:00
   W003      CT_Scan_Chest         1   pending        None 2024-11-10 13:13:00
   W004      CT_Scan_Brain         3   pending        None 2024-11-12 04:34:00
   W005        Mammography         3  assigned        R015 2024-11-10 08:48:00
   W006 Ultrasound_Abdomen         2   pending        None 2024-11-10 21:48:00
   W007      CT_Scan_Chest         1   pending        None 2024-11-11 06:38:00
   W008      CT_Scan_Brain         2   pending        None 2024-11-11 08:05:00
   W009         X_Ray_Bone         3   pending        None 2024-11-10 10:42:00
   W010        X_Ray_Chest         3   pending        None 2024-11-10 14:24:00
   W011         X_Ray_Bone         2   pending        None 2024-11-10 21:42:00

In [12]:
# Clean up
db.close()
print("✅ Database connection closed")
print("\n🎉 Fixed implementation complete with true LangGraph orchestration!")

✅ Database connection closed

🎉 Fixed implementation complete with true LangGraph orchestration!
